In [32]:
import pandas as pd
import db_access
import math

In [33]:
def get_raw_data(query):
    engine = db_access.get_engine()
    return pd.read_sql(query, engine)

In [34]:
query = "select * from usd_jpy_15m where datetime between '2019/08/01' and '2020/02/01'"

df = get_raw_data(query)
df = df[["datetime","open","high","low","close","volume"]].copy()
df.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']
df['Datetime'] = pd.to_datetime(df['Datetime'])
df = df.reset_index().set_index('Datetime')
df.head()

,index,Open,High,Low,Close,Volume
Datetime,,,,,,
2019-08-01 00:00:00,0,108.824,108.904,108.812,108.885,367
2019-08-01 00:15:00,1,108.883,109.176,108.880,109.131,926
2019-08-01 00:30:00,2,109.128,109.222,109.123,109.213,540
2019-08-01 00:45:00,3,109.210,109.320,109.199,109.258,667
2019-08-01 01:00:00,4,109.255,109.290,109.212,109.256,440


In [35]:
df.tail()

,index,Open,High,Low,Close,Volume
Datetime,,,,,,
2020-01-31 20:45:00,12447,108.402,108.420,108.341,108.352,3090
2020-01-31 21:00:00,12448,108.354,108.374,108.343,108.369,657
2020-01-31 21:15:00,12449,108.370,108.382,108.338,108.338,258
2020-01-31 21:30:00,12450,108.337,108.364,108.335,108.363,174
2020-01-31 21:45:00,12451,108.364,108.372,108.354,108.358,150


In [36]:
import talib as ta

In [37]:
def get_bband(close, timeperiod=20, nbdevup=3, nbdevdn=3, matype=0):
    gain = pd.DataFrame(close)
    gain.columns = ['close']
    upper, middle, lower = ta.BBANDS(gain.close, timeperiod, nbdevup , nbdevdn, matype)
    gain['bb_upper'] = upper
    gain['bb_lower'] = lower
    return gain['bb_upper'], gain['bb_lower']

In [38]:
def get_macd(close, fastperiod=6, slowperiod=13, signalperiod=4):
    macd, macdsignal, macdhist = ta.MACD(close, fastperiod=fastperiod, slowperiod=slowperiod, signalperiod=signalperiod)
    return macd, macdsignal

def get_macd_with_hist(close, fastperiod=6, slowperiod=13, signalperiod=4):
    macd, macdsignal, macdhist = ta.MACD(close, fastperiod=fastperiod, slowperiod=slowperiod, signalperiod=signalperiod)
    return macd, macdsignal, macdhist

In [39]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

In [40]:
class MacdBbandStrategy(Strategy):
    macd_fastperiod = 12
    macd_slowperiod = 26
    macd_signalperiod = 9
    profit_pips = 30
    loss_pips = 15
    profit = profit_pips * 0.01
    loss = loss_pips * 0.01
    hist_seps = 2
    day_seps = 10
    
    def init(self):
        self.macd, self.signal, self.hist = self.I(get_macd_with_hist, self.data.Close, self.macd_fastperiod, self.macd_slowperiod, self.macd_signalperiod)
        
        # 乖離してからのロウソク数
        self.seps = 0

    def next(self):
        # 乖離確認
        if (math.fabs(self.hist) * 100 > self.hist_seps):
            self.seps = self.day_seps

        if (self.seps > 0):
            if (crossover(self.signal, self.macd)):
                if (self.signal[-2] < self.macd[-2]):
                    self.buy(sl=self.data.Close[-1] - self.loss, tp=self.data.Close[-1] + self.profit)
                elif (self.signal[-2] > self.macd[-2]):
                    self.sell(sl=self.data.Close[-1] + self.loss, tp=self.data.Close[-1] - self.profit)
            else:
                self.seps -= 1


In [41]:
bt = Backtest(df, MacdBbandStrategy, cash=100000, commission=.00004)
output = bt.run()
print(output)

Start                     2019-08-01 00:00:00
End                       2020-01-31 21:45:00
Duration                    183 days 21:45:00
Exposure [%]                          15.8652
Equity Final [$]                      99152.8
Equity Peak [$]                        100126
Return [%]                          -0.847244
Buy & Hold Return [%]                0.483997
Max. Drawdown [%]                    -1.54423
Avg. Drawdown [%]                   -0.766206
Max. Drawdown Duration      148 days 16:30:00
Avg. Drawdown Duration       45 days 13:57:00
# Trades                                   60
Win Rate [%]                          31.6667
Best Trade [%]                       0.283956
Worst Trade [%]                     -0.204444
Avg. Trade [%]                     -0.0140135
Max. Trade Duration           2 days 20:45:00
Avg. Trade Duration           0 days 11:41:00
Expectancy [%]                       0.184035
SQN                                 -0.565188
Sharpe Ratio                      

In [42]:
stats = bt.optimize(
    macd_fastperiod = [8, 10, 12],
    macd_slowperiod = [13, 19, 26],
    macd_signalperiod = [4, 7, 9],
    profit_pips = [15, 30],
    loss_pips = [10, 20],
    hist_seps = [1, 2],
    day_seps = [6, 8, 10]
)

/Users/kataoka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Searching best of 648 configurations.
  


In [43]:
print(stats)

Start                                                   2019-08-01 00:00:00
End                                                     2020-01-31 21:45:00
Duration                                                  183 days 21:45:00
Exposure [%]                                                       0.708015
Equity Final [$]                                                     100961
Equity Peak [$]                                                      100961
Return [%]                                                         0.961151
Buy & Hold Return [%]                                              0.483997
Max. Drawdown [%]                                                 -0.320126
Avg. Drawdown [%]                                                -0.0892271
Max. Drawdown Duration                                     40 days 18:00:00
Avg. Drawdown Duration                                      4 days 10:17:00
# Trades                                                                  8
Win Rate [%]

In [20]:
print(stats._strategy)

MacdBbandStrategy2(macd_fastperiod=12,macd_slowperiod=26,macd_signalperiod=4,profit_pips=15,loss_pips=10,hist_seps=2,day_seps=10)


In [31]:
bt.plot()